# PBWDeformations.jl examples

We present the current functionality of the PBWDeformations.jl package.

In [1]:
using Oscar, PBWDeformations

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.7.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2021 by The Oscar Development Team


In [2]:
] status

     Project PBWDeformations v0.0.0
      Status `~/code/pbwdeformations.jl/Project.toml`
  [c3fe647b] AbstractAlgebra v0.22.2
  [861a8166] Combinatorics v1.0.2
  [f1435218] Oscar v0.7.0
  [2f01184e] SparseArrays


## Smash products

One can create smash products of the form $$TV \rtimes H$$ where $H = U(L)$ for some finite-dimensional semisimple Lie algebra $L$ and $V$ some finite dimensional highest weight $L$-module.
Currently, there are only the classical types of lie algebras over $\mathbb{Q}$ supported.

As an ongoing example, we use the smash product corresponding dynkin type $A_1$ (i.e. $\mathfrak{sl}_3(\mathbb{Q})$) and the highest weight vector $\lambda = (1)$ (i.e. the natural representation).

In [3]:
sp, ((x_1, x_2, x_3), (v_1, v_2)) = PBWDeformations.smash_product_lie(QQ, 'A', 1, [1])

(Lie Algebra Smash Product with basis x_1, x_2, x_3, v_1, v_2 over Rational Field, (PBWDeformations.QuadraticQuoAlgebraElem{fmpq}[x_1, x_2, x_3], PBWDeformations.QuadraticQuoAlgebraElem{fmpq}[v_1, v_2]))

`sp` is a struct containing some data. Of particular interest is `sp.alg`, the underlying algebra that implements most of the NCRing interface of `AbstractAlgebra.jl`.

On can check that the basis elements' commutators are as expected, i.e. $[x_i,x_j] = [x_i,x_j]_L$ and $[x_i,v_k] = x_i \cdot v_k$.

In [4]:
@assert comm(x_1, x_2; strict=true) == x_3      # [x,y]   =   h  = [x,y]_L
@assert comm(x_3, x_1; strict=true) == 2*x_1    # [h,x]   =  2x  = [h,x]_L
@assert comm(x_3, x_2; strict=true) == -2*x_2   # [h,y]   = -2y  = [h,y]_L

@assert comm(x_1, v_1; strict=true) == 0        # [x,v_1] =  0   = x⋅v_1
@assert comm(x_2, v_1; strict=true) == v_2      # [y,v_1] =  v_2 = y⋅v_1
@assert comm(x_3, v_1; strict=true) == v_1      # [h,v_1] =  v_1 = h⋅v_1
@assert comm(x_1, v_2; strict=true) == v_1      # [x,v_2] =  v_1 = x⋅v_2
@assert comm(x_2, v_2; strict=true) == 0        # [y,v_2] =  0   = y⋅v_2
@assert comm(x_3, v_2; strict=true) == -v_2     # [h,v_2] = -v_2 = h⋅v_2

The above used optional parameter `strict` forces the computation of a normal form in a PBW basis and is present in the functions `isequal`, `iszero`, `isone` and `comm`. The default implementation of these methods interpretes its arguments as elements of a suitable free associative algebra, and thus does not apply any relations to reduce the computational effort if it is not needed.  
Besides in the strict versions of these methods, one can compute the normal form of any expression directly.

In [5]:
a = v_2 + x_2^2 * x_1 - 3*v_2

-2*v_2 + x_2^2*x_1

In [6]:
normal_form(a)

-2*v_2 + -2*x_2*x_3 + x_1*x_2^2 + 2*x_2

## Deforming smash products

A deformation of a smash product $TV \rtimes H$ is formally $$A := A_{H,V,\kappa} := (TV \rtimes H)/I_\kappa$$ where $I_\kappa := \big([v_i,v_j] - \kappa(v_i \wedge v_j)\big)$ for some morphism $\kappa: V \wedge V \to H$.

In the context of this package, $\kappa$ is always represented as a matrix $M_\kappa$ where $M_\kappa[i,j] = \kappa(v_i \wedge v_j)$.

As a first example, consider the symmetric deformation induced by $\kappa = 0$ and denoted by $A_0 := A_{H,V,0}$.

In [7]:
PBWDeformations.smash_product_symmdeform_lie(sp)

(Symmetric Deformation of Lie Algebra Smash Product with basis x_1, x_2, x_3, v_1, v_2 over Rational Field, (PBWDeformations.QuadraticQuoAlgebraElem{fmpq}[x_1, x_2, x_3], PBWDeformations.QuadraticQuoAlgebraElem{fmpq}[v_1, v_2]))

This is in fact only a shorthand for the following, slightly longer code.

In [8]:
kappa1 = fill(zero(sp.alg), sp.dimV, sp.dimV)
deform1, _ = PBWDeformations.smash_product_deform_lie(sp, kappa1)

(Symmetric Deformation of Lie Algebra Smash Product with basis x_1, x_2, x_3, v_1, v_2 over Rational Field, (PBWDeformations.QuadraticQuoAlgebraElem{fmpq}[x_1, x_2, x_3], PBWDeformations.QuadraticQuoAlgebraElem{fmpq}[v_1, v_2]))

One can make $\kappa$ arbitrarily complicated, as long as $M_\kappa$ is skew-symmetric. The second, here suppressed, return value is again a tuple of lists of generators for $U(L)$ and $TV$ (cf. [`smash_product_lie`](#Smash-products)), now respecting the relations from $I_\kappa$ in a `strict=true` context as well.

In [9]:
kappa2 = fill(zero(sp.alg), sp.dimV, sp.dimV)
kappa2[1,2] = x_1^2 + 5*x_2*x_3 + 3
kappa2[2,1] = -kappa2[1,2]
deform2, _ = PBWDeformations.smash_product_deform_lie(sp, kappa2)

(Deformation of Lie Algebra Smash Product with basis x_1, x_2, x_3, v_1, v_2 over Rational Field, (PBWDeformations.QuadraticQuoAlgebraElem{fmpq}[x_1, x_2, x_3], PBWDeformations.QuadraticQuoAlgebraElem{fmpq}[v_1, v_2]))

In [10]:
kappa3 = fill(zero(sp.alg), sp.dimV, sp.dimV)
kappa3[1,2] = 4*x_1*x_2 + x_3^2 - 2*x_3 + 42
kappa3[2,1] = -kappa3[1,2]
deform3, _ = PBWDeformations.smash_product_deform_lie(sp, kappa3)

(Deformation of Lie Algebra Smash Product with basis x_1, x_2, x_3, v_1, v_2 over Rational Field, (PBWDeformations.QuadraticQuoAlgebraElem{fmpq}[x_1, x_2, x_3], PBWDeformations.QuadraticQuoAlgebraElem{fmpq}[v_1, v_2]))

In [11]:
kappa4 = fill(zero(sp.alg), sp.dimV, sp.dimV)
kappa4[1,2] = x_2^2
kappa4[2,1] = -x_1
deform4, _ = PBWDeformations.smash_product_deform_lie(sp, kappa4)

LoadError: ArgumentError: kappa is not skew-symmetric.

## PBW deformations

$A$ becomes a filtered algebra via $$F_i(A) = \overline{T^{\leq i}V \rtimes H}.$$
A deformation $A$ is called a *PBW deformation* of $A_0 = SV \rtimes H$ if $\mathop{gr} A \cong A_0$ (as $\mathbb{N}$-graded algebras).

This package can check for any $\kappa$ if this induces a PBW deformation. To achieve this, it uses Theorem 3.1 of [[1]](#References).

In [12]:
PBWDeformations.ispbwdeform(deform1)

true

The symmetric deformation is *always* a PBW deformation.

In [13]:
PBWDeformations.ispbwdeform(deform2)

false

Most deformations are no PBW deformations.

In [14]:
PBWDeformations.ispbwdeform(deform3)

true

But sometimes they are.

## Computing *all* PBW deformations

Using Theorem 3.1 of [[1]](#References), one can compute a basis of the matrix space of all $M_\kappa$ that induce a PBW deformation. The second parameter to the below function sets the maximal degree of all elements in $M_\kappa$.

**WARNING**: This computation needs a lot of time and RAM, even for small examples.

In [15]:
kappas = PBWDeformations.possible_pbwdeforms(sp, 4)

┌ Info: Constructing MPolyRing...
└ @ PBWDeformations /home/lgoe/code/pbwdeformations.jl/src/SmashProductDeformLie.jl:242
┌ Info: Changing SmashProductLie coeffcient type...
└ @ PBWDeformations /home/lgoe/code/pbwdeformations.jl/src/SmashProductDeformLie.jl:249
┌ Info: Constructing kappa...
└ @ PBWDeformations /home/lgoe/code/pbwdeformations.jl/src/SmashProductDeformLie.jl:252
┌ Info: Constructing deformation...
└ @ PBWDeformations /home/lgoe/code/pbwdeformations.jl/src/SmashProductDeformLie.jl:259
┌ Info: Generating equation iterator...
└ @ PBWDeformations /home/lgoe/code/pbwdeformations.jl/src/SmashProductDeformLie.jl:262
┌ Info: Computing row-echelon form...
└ @ PBWDeformations /home/lgoe/code/pbwdeformations.jl/src/SmashProductDeformLie.jl:278
┌ Info: Computing reduced row-echelon form...
└ @ PBWDeformations /home/lgoe/code/pbwdeformations.jl/src/SmashProductDeformLie.jl:284
┌ Info: Computing a basis...
└ @ PBWDeformations /home/lgoe/code/pbwdeformations.jl/src/SmashProductDeformLi

3-element Vector{Matrix{PBWDeformations.QuadraticQuoAlgebraElem{fmpq}}}:
 [0 1; -1 0]
 [0 -2*x_3 + 4*x_1*x_2 + x_3^2; 2*x_3 + -4*x_1*x_2 + -1*x_3^2 0]
 [0 8*x_3 + 16*x_1*x_2 + -32*x_1*x_2*x_3 + -4*x_3^3 + 16*x_1^2*x_2^2 + 8*x_1*x_2*x_3^2 + x_3^4; -8*x_3 + -16*x_1*x_2 + 32*x_1*x_2*x_3 + 4*x_3^3 + -16*x_1^2*x_2^2 + -8*x_1*x_2*x_3^2 + -1*x_3^4 0]

The space of $M_\kappa$ of degree at most 4 and inducing a PBW deformation has dimension 3.

We can check that these indeed induce PBW deformations.

In [16]:
for kappa in kappas
    d, _ = PBWDeformations.smash_product_deform_lie(sp, kappa)
    println(PBWDeformations.ispbwdeform(d))
end

true
true
true


## References

[1] Walton, Witherspoon: Poincare-Birkhoff-Witt deformations of smash product algebras from Hopf actions on Koszul algebras. DOI: 10.2140/ant.2014.8.1701. https://arxiv.org/abs/1308.6011